In [2]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

In [4]:
# From: http://localhost:8889/notebooks/PycharmProjects/neutral_loss/nl_0_3_clean_join_nb.ipynb
# Load dataframe of previously identified high-quality annotations
hq_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/high_quality_cm3_output_02.pickle')

In [12]:
# Filter for pertinent columns and fdr better than 0.2
good_col = ['ds_id',
            'hmdb_ids',
            'polarity',
            'adduct',
            'formula',
            'fdr',
            'group',
            'analyzer'
           ]
hq_df = hq_df[good_col].copy(deep=True)
hq_df = hq_df[hq_df.fdr <= 0.2]
hq_df['polarity'].replace(1, 'positive', inplace=True)
hq_df['polarity'].replace(-1, 'negative', inplace=True)

In [13]:
# Keep for merging downstream with other datasets
good_col2 = ['ds_id',
            'polarity',
            'group',
            'analyzer'
           ]
hq_ds_ids = hq_df[good_col2].copy(deep=True)
hq_ds_ids.drop_duplicates(keep="first", inplace=True)
hq_ds_ids = hq_ds_ids.reset_index(drop=True)
hq_ds_ids['expt_type'] = 'high_quality'
hq_ds_ids

,ds_id,polarity,group,analyzer,expt_type
0,2017-08-01_07h47m38s,positive,U Rennes 1,FTICR,high_quality
1,2017-08-07_14h26m32s,positive,MPI Bremen,Orbitrap,high_quality
2,2017-02-17_20h48m57s,positive,♡EMBL♡,Orbitrap,high_quality
3,2017-07-21_14h46m19s,positive,U Rennes 1,FTICR,high_quality
4,2016-12-01_18h42m02s,positive,U Melbourne,FTICR,high_quality
...,...,...,...,...,...
313,2019-06-25_21h52m46s,negative,UTHSA,Orbitrap,high_quality
314,2019-06-21_08h59m52s,positive,MPI Bremen,Orbitrap,high_quality
315,2017-12-07_17h00m26s,positive,♡EMBL♡,Orbitrap,high_quality
316,2019-06-25_16h15m15s,positive,MPI Bremen,Orbitrap,high_quality


In [14]:
hq_ds_ids = pd.merge(hq_ds_ids, out_df, how='inner', 
                     left_on='ds_id', right_on='ds_id_in')

In [16]:
hq_ds_ids.to_pickle('combined_analysis/hq_ds_ids.pickle')

In [ ]:
# Paths to databases and output for next cell
limit_list = None
outname = ''
out_path = 'to_metaspace_hq/'
polarity = None
expt_neg = 'test_output/exp_negative.pickle'
expt_pos = 'test_output/exp_positive.pickle'
theo_neg = 'test_output/theo_negative.pickle'
theo_pos = 'test_output/theo_positive.pickle'
ref_expt = 'databases/ref_expt_df.pickle'
ref_theo = 'databases/ref_theo_df.pickle'

In [ ]:
# Generats METASPACE database from Sirius output files
from sirius_to_metaspace_db import primary_loop

ds_id_list = list(hq_df.ds_id.unique())
for ds in ds_id_list:
    polarity = list(hq_df[hq_df.ds_id == ds].polarity.unique())[0]
    id_list = list(df[df.ds_id == ds].hmdb_ids.unique())
    print('ds: ', ds, ' ds_id_list: ', len(ds_id_list), 
          ' polarity: ', polarity)
    primary_loop(id_list,
                 ds, 
                 out_path,
                 polarity,
                 expt_pos,
                 theo_pos,
                 expt_neg,
                 theo_neg,
                 ref_expt,
                 ref_theo
                )

In [ ]:
# Generates entire METASPACE_MSMS database
from sirius_to_metaspace_db import primary_loop


primary_loop([],
             'cm3_MSMS_positive_all.pickle', 
             'to_metaspace/',
             'positive',
             expt_pos,
             theo_pos,
             expt_neg,
             theo_neg,
             ref_expt,
             ref_theo
            )

primary_loop([],
             'cm3_MSMS_negative_all.pickle', 
             'to_metaspace/',
             'negative',
             expt_pos,
             theo_pos,
             expt_neg,
             theo_neg,
             ref_expt,
             ref_theo
            )

In [ ]:
# Copy dataset (beta to beta) and search with custom METASPACE MSMS db
from clone_ds_to_beta import copy_beta

ds_id_list = list(hq_df.ds_id.unique())[1:]
out_dict = {}
for ds_id in ds_id_list:
    print(ds_id)
    out_dict[ds_id] = copy_beta(ds_id, 'M', ds_id, prod_to_beta=False)

In [ ]:
out_df = pd.DataFrame(out_dict).T.reset_index()
out_df['search'] = 'HQ_data_msms'
out_df = out_df.iloc[:,1:]
out_df.to_pickle('2020_May_18_HQ_dataset_msms_ds_ids.csv')

In [9]:
out_df = pd.read_pickle('2020_May_18_HQ_dataset_msms_ds_ids.csv')
out_df

,ds_id_in,ds_id_out,search
0,2017-08-07_14h26m32s,2020-05-18_17h00m04s,HQ_data_msms
1,2017-02-17_20h48m57s,2020-05-18_17h00m06s,HQ_data_msms
2,2017-07-21_14h46m19s,2020-05-18_17h00m08s,HQ_data_msms
3,2016-12-01_18h42m02s,2020-05-18_17h00m11s,HQ_data_msms
4,2017-08-08_08h49m46s,2020-05-18_17h00m13s,HQ_data_msms
...,...,...,...
312,2019-06-25_21h52m46s,2020-05-18_17h12m09s,HQ_data_msms
313,2019-06-21_08h59m52s,2020-05-18_17h12m11s,HQ_data_msms
314,2017-12-07_17h00m26s,2020-05-18_17h12m14s,HQ_data_msms
315,2019-06-25_16h15m15s,2020-05-18_17h12m16s,HQ_data_msms


In [27]:
# Weird error, getting index and value back from series with one entry?
ds_db_dict = dict(zip(list(out_df.ds_id_in), list(out_df.ds_id_in)))
ds_ms_dict = dict(zip(list(out_df.ds_id_in), list(out_df.ds_id_out)))

In [28]:
from results_from_metaspace_msms_process import reporting_loop

for ori_ds, db in ds_db_dict.items():
    ds_ms = ds_ms_dict[ori_ds]
    
    reporting_loop(ori_ds,
                   db,
                   ds_ms,
                   'METASPACE_img_df_2020_May_18_good_quality_arr/',
                   parent_and_fragment_req=True,
                   fdr_max=0.5,
                   save_image=False)

2020-05-18_17h08m38s  Counter:  3470
2020-05-18_17h08m40s  Counter:  2669
2020-05-18_17h08m42s  Counter:  1395
2020-05-18_17h08m44s  Counter:  250
2020-05-18_17h08m47s  Counter:  4203
2020-05-18_17h08m49s  Counter:  3226
2020-05-18_17h08m51s  Counter:  9293
2020-05-18_17h08m54s  Counter:  633
2020-05-18_17h08m56s  Counter:  2239
2020-05-18_17h08m58s  Counter:  214
2020-05-18_17h09m01s  Counter:  4154
2020-05-18_17h09m03s  Counter:  561
2020-05-18_17h09m05s  Counter:  4404
2020-05-18_17h09m07s  Counter:  3789
2020-05-18_17h09m10s  Counter:  536
2020-05-18_17h09m13s  Counter:  17
2020-05-18_17h09m15s  Counter:  3797
2020-05-18_17h09m17s  Counter:  3269
2020-05-18_17h09m19s  Counter:  3276
2020-05-18_17h09m21s  Counter:  518
2020-05-18_17h09m24s  Counter:  752
2020-05-18_17h09m27s  Counter:  3689
2020-05-18_17h09m30s  Counter:  252
2020-05-18_17h09m33s  Counter:  3866
2020-05-18_17h09m35s  Counter:  4643
2020-05-18_17h09m37s  Counter:  15
2020-05-18_17h09m39s  Counter:  3417
2020-05-18_17

In [ ]:
Thursday:
# Merge hq_ds, spotting_ds, and wb_ds
# cm3 db for calculating isobar intersections, also do so in each ds_id
# frame work for different groups on ds_id, join 9x spotted and high_quality
# Code new scoring functions
# SpaceM to Andreas

Future:
# To compare expt only versus expt/theo

